In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd

/home/jk/enter/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [9]:
sentence = [
    'python python python java java', 
    'python python java html',
    'python html html html css', 
       
]

In [10]:
# countvectorizer
cv = CountVectorizer()

In [11]:
cv_obj = cv.fit_transform(sentence)

In [12]:
cv.get_feature_names()

['css', 'html', 'java', 'python']

In [16]:
cv_df = pd.DataFrame(cv_obj.todense(), columns = cv.get_feature_names())
cv_df

,css,html,java,python
0,0,0,2,3
1,0,1,1,2
2,1,3,0,1


In [18]:
# TfidfVectorizer
tv = TfidfVectorizer()

In [19]:
tv_obj = tv.fit_transform(sentence)

In [21]:
tv_df = pd.DataFrame(tv_obj.toarray(), columns=tv.get_feature_names())
tv_df

,css,html,java,python
0,0.0000,0.000000,0.651365,0.758765
1,0.0000,0.476063,0.476063,0.739411
2,0.3906,0.891185,0.000000,0.230695


In [23]:
tv_similarity = pd.DataFrame((tv_obj*tv_obj.T).toarray(), columns=tv_df.index)
tv_similarity

,0,1,2
0,1.000000,0.871129,0.175043
1,0.871129,1.000000,0.594839
2,0.175043,0.594839,1.000000


In [24]:
tv.idf_

array([1.69314718, 1.28768207, 1.28768207, 1.        ])

In [66]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import pickle
from konlpy.tag import Twitter

In [31]:
article_df = pd.read_pickle('article_2016-06-01.plk')
print(len(article_df))
article_df.head()

2327


,newsid,oid,newspaper,title,link,comment,likeit,content,date,category
0,3689326,8,머니투데이,美 '北 주요자금세탁우려대상국' 지정…'돈줄' 차단 강화,http://news.naver.com/main/read.nhn?mode=LSD&m...,5,7,[머니투데이 뉴욕=서명훈 특파원] 미국이 1일(현지시간) 북한을 '주요 자금세탁 우...,2016-06-01,0
1,8444786,1,연합뉴스,"美재무부, 북한 '주요 자금세탁 우려 대상국' 첫 지정(3보)",http://news.naver.com/main/read.nhn?mode=LSD&m...,64,35,미 재무부 청사<<연합뉴스 DB>>국제금융망 접근 차단…국제사회에도 북한과 금융거래...,2016-06-01,0
2,3556076,18,이데일리,'냉정'과 '대화' 강조한 시진핑…北뿐 아니라 한미일도 겨냥,http://news.naver.com/main/read.nhn?mode=LSD&m...,12,2,- 유관국의 냉정과 자제는 北 뿐 아니라 6자회담 당사국 모두 향한 메시지- 대화 ...,2016-06-01,0
3,8444772,1,연합뉴스,"北, '200일 전투' 평양시 군중대회 개최",http://news.naver.com/main/read.nhn?mode=LSD&m...,16,2,"北,'200일 전투' 평양시 군중대회 개최 (서울=연합뉴스) 북한이 올해 두 ...",2016-06-01,0
4,8444770,1,연합뉴스,"시진핑, 北리수용에 &quot;냉정·자제…지역 평화·안정 수호해야&quot;(종합2보)",http://news.naver.com/main/read.nhn?mode=LSD&m...,3,1,"시진핑, 리수용과 면담(베이징=연합뉴스) 시진핑(習近平) 중국 국가주석이 1일 베이...",2016-06-01,0


### split train, test

In [35]:
X_train, X_test, y_train, y_test = train_test_split(article_df.content, article_df.category ,test_size = 0.1, random_state=1)

In [82]:
# pos_tagger = Twitter()
# def tokenize_pos(doc):
#     return ["/".join(t) for t in pos_tagger.pos(doc)]

In [ ]:
# clf = Pipeline([
#     ('vect', TfidfVectorizer(tokenizer=tokenize_pos)),
#     ('clf', MultinomialNB(alpha=0.01)),   
# ])

In [83]:
clf = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultinomialNB(alpha=0.01)),   
])

In [84]:
model = clf.fit(X_train, y_train)

### confusion matrix, classification report

In [85]:
from sklearn.metrics import confusion_matrix, classification_report

In [86]:
y_pred = model.predict(X_test)

In [87]:
confusion_matrix(y_pred, y_test)

array([[41,  1,  1,  0,  1,  0],
       [ 0, 33,  2,  0,  1,  1],
       [ 3,  0, 77,  2,  2,  1],
       [ 1,  5,  2, 18,  0,  0],
       [ 1,  1,  1,  0, 25,  0],
       [ 0,  1,  0,  1,  0, 11]])

In [88]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.93      0.89      0.91        46
          1       0.89      0.80      0.85        41
          2       0.91      0.93      0.92        83
          3       0.69      0.86      0.77        21
          4       0.89      0.86      0.88        29
          5       0.85      0.85      0.85        13

avg / total       0.88      0.88      0.88       233



# prediction

In [89]:
X_test = X_test.reset_index(drop=True)
X_test[0][:50], X_test[1][:50], X_test[2][:50]

('인천 송도 삼성바이오로직스 현장을 가다“삼성이 처음 핸드폰 사업을 시작할 때 망치로 핸드폰',
 '[구의역 사고를 보며] 청년을 위한 나라는 없다\xa0[ 주동식 지역평등시민연대 대표]\xa0요즘 사',
 ' [한겨레] 구의역·남양주 사고에 입법대책 분주더민주, 직접 고용법 제정안 발의국민의당, ')

In [90]:
result = model.predict([X_test[0], X_test[1], X_test[2]])
result

array([1, 2, 0])

In [91]:
classification_dict = {
    0:'정치', 1:'경제', 2:'사회',3:'생활/문화', 4:'세계', 5:'IT/과학'
        
}
for idx, category in enumerate(result):
    print(classification_dict[category], '-', X_test[idx][:50])

경제 - 인천 송도 삼성바이오로직스 현장을 가다“삼성이 처음 핸드폰 사업을 시작할 때 망치로 핸드폰
사회 - [구의역 사고를 보며] 청년을 위한 나라는 없다 [ 주동식 지역평등시민연대 대표] 요즘 사
정치 -  [한겨레] 구의역·남양주 사고에 입법대책 분주더민주, 직접 고용법 제정안 발의국민의당, 


### save and load model

In [92]:
pickle.dump(model, open('classification.plk', 'wb'))

In [93]:
load_model = pickle.load(open('classification.plk', 'rb'))

In [94]:
str1 = '네이버와 카카오는 드론 기술 발전에 주력'
str2 = '요즘 환률과 주가는 예측이 불가'
load_model.predict([str1, str2])

array([5, 0])

In [95]:
classification_dict[load_model.predict([str1])[0]], classification_dict[load_model.predict([str2])[0]]

('IT/과학', '정치')

In [96]:
load_model.predict_proba([str2])

array([[0.38518805, 0.35692534, 0.12105192, 0.04011175, 0.03158595,
        0.065137  ]])